In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score,ConfusionMatrixDisplay


os.environ['TF_FORCE_UNIFIED_MEMORY']='1'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='2.0'
#os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH ']='true' # as I understood, this is redundant with the set_memory_growth part :)

import tensorflow as tf    
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      print(gpu)
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import keras.backend as K
from keras.optimizers import Adam

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack



import pickle
from gensim.models import Word2Vec

def load_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".c"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as file_content:
                    file_text = file_content.read()
                file_list.append(file_text)
    return file_list
# Load training and test files for vulnerable and non-vulnerable classes
def load_data():
    dataset_dir = 'F://Development_//Deep Learning//Basic Dataset//BTP//1//Dataset//Raw//dataset 1//Dataset_raw'
    vulnerable_dir = os.path.join(dataset_dir, 'Vulnerable')
    non_vulnerable_dir = os.path.join(dataset_dir, 'Non_vulnerable')

    vulnerable_train_files = load_files(os.path.join(vulnerable_dir, 'TRAIN'))
    vulnerable_test_files = load_files(os.path.join(vulnerable_dir, 'TEST'))
    non_vulnerable_train_files = load_files(os.path.join(non_vulnerable_dir, 'TRAIN'))
    non_vulnerable_test_files = load_files(os.path.join(non_vulnerable_dir, 'TEST'))

    train_codes = vulnerable_train_files + non_vulnerable_train_files
    test_codes = vulnerable_test_files + non_vulnerable_test_files

    # train_codes = train_codes+ test_codes

    labels = np.concatenate((np.ones(len(vulnerable_train_files)), np.zeros(len(non_vulnerable_train_files))), axis=0)

    X_train, X_test, y_train, y_test = train_test_split(train_codes, labels, test_size=0.3, random_state=42)

    # Print lengths of all datasets
    print("Length of X_train:", len(X_train))
    print("Length of X_test:", len(X_test))
    print("Length of y_train:", len(y_train))
    print("Length of y_test:", len(y_test))
    print("Length of test_codes:", len(test_codes))



    return X_train, X_test, y_train, y_test, test_codes


def tokenize_sequences(X_train, X_test, test_codes):
    tokenizer = Tokenizer(num_words=359414, split=' ')
    tokenizer.fit_on_texts(X_train)
    X_train_sequences = tokenizer.texts_to_sequences(X_train)
    X_test_sequences = tokenizer.texts_to_sequences(X_test)
    test_sequences = tokenizer.texts_to_sequences(test_codes)

    average_sequence_length = int(np.mean([len(seq) for seq in X_train_sequences]))

    X_train_padded = pad_sequences(X_train_sequences, maxlen=average_sequence_length)
    X_test_padded = pad_sequences(X_test_sequences, maxlen=average_sequence_length)
    test_padded = pad_sequences(test_sequences, maxlen=average_sequence_length)

    return X_train_padded, X_test_padded, test_padded, tokenizer.word_index



In [ ]:
import os
from keras.models import load_model
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

import sys
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt

import os

def load_all_models(i=0):
    all_models = list()

    folder_path = 'F://Development_//Deep Learning//Basic Dataset//BTP//1//Outputs//Models//LSTM//word2vec//'
    model_files = [f for f in os.listdir(folder_path) if f.endswith('.keras')]
    
    for filename in model_files:
        model_path = os.path.join(folder_path, filename)
        model = load_model(model_path)
        all_models.append(model)
        print(f'> Loaded {model_path}')

    return all_models


from sklearn.linear_model import SGDClassifier

def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = np.dstack((stackX, yhat))
    
    print("Sample of stackX before reshaping:")
    for i in range(min(3, stackX.shape[0])):
        print(np.round(stackX[i], decimals=4))
        
    # flatten predictions to [rows, members x probabilities]
    print("Shape of stackX before reshaping:", stackX.shape)
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    print("Shape of stackX after reshaping:", stackX.shape)
    
    print("Sample of stackX after reshaping:")
    for i in range(min(3, stackX.shape[0])):
        print(np.round(stackX[i], decimals=4))

    return stackX



def fit_stacked_model(members, inputX, inputy, validation_split=0.1):
    stackedX = stacked_dataset(members, inputX)
    
    model = Sequential([
        Dense(128, activation='relu', input_shape=(stackedX.shape[1],)),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(stackedX, inputy, epochs=50, batch_size=64, verbose=1, validation_split=validation_split)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return model




# def evaluate_model(X_test_padded, y_test, model):
#     stackedX = stacked_dataset(members, inputX)
#     y_pred_prob = model.predict(X_test_padded)
#     y_pred = (y_pred_prob > 0.5).astype(int)
#     report = classification_report(y_test, y_pred)
#     confusion = confusion_matrix(y_test, y_pred)
#     class_names = {0: "Non-vulnerable", 1: "Vulnerable"}  # Example mapping for binary classes
#     labels = list(class_names.values())

#     disp = ConfusionMatrixDisplay(confusion_matrix = confusion, display_labels = labels)
#     disp.plot(cmap=plt.cm.Blues)
#     plt.show()
#     print(report)
#     print("Confusion Matrix:")
#     print(confusion)
    
def stacked_prediction(members, model, inputX, y_test, threshold=0.5):
    stackedX = stacked_dataset(members, inputX)
    yhat = model.predict(stackedX)
    yhat_binary = (yhat > threshold).astype(int)
    report = classification_report(y_test, yhat_binary)

    confusion = confusion_matrix(y_test, yhat_binary)
    class_names = {0: "Non-vulnerable", 1: "Vulnerable"}  # Example mapping for binary classes
    labels = list(class_names.values())

    tn, fp, fn, tp = confusion.ravel()
    fpr = fp / (fp + tn)
    tpr = tp / (fn + tp)
    
    disp = ConfusionMatrixDisplay(confusion_matrix = confusion, display_labels = labels)
    disp.plot(cmap=plt.cm.Blues)
    plt.show()
    print(report)
    print("Confusion Matrix:")
    print(confusion)
    print("False Positive Rate (FPR):", fpr)
    print("False Negative Rate (TPR):", tpr)
    
    return yhat_binary

X_train, X_test, y_train, y_test, test_codes = load_data()

# Tokenize the sequences
X_train_padded, X_test_padded, test_padded, word_index = tokenize_sequences(X_train, X_test, test_codes)

import itertools

members = load_all_models(1)
# for model in members:
#    _, acc = model.evaluate(X_test_padded, y_test, verbose=0)
#    print('Model Accuracy: %.3f' % acc

# fit stacked model using the ensemble
model = fit_stacked_model(members, X_train_padded, y_train)

# evaluate model on test set
yhat = stacked_prediction(members, model, X_test_padded, y_test)
print("Sample of y_test:")
print(y_test[:5])  # Print the first 5 elements of y_test

print("Sample of yhat:")
print(yhat[:5])  # Print the first 5 elements of yhat


acc = accuracy_score(y_test, yhat)
print('Stacked Test Accuracy: %.3f' % acc)
# evaluate_model(X_test_padded, y_test, model)
